# Automated Address Lookup and Geocoding



In [1]:
! pip install --upgrade google-maps-places
! pip install --upgrade geopandas
! pip install --upgrade fiona

     |████████████████████████████████| 74 kB 5.0 MB/s  eta 0:00:01
     |████████████████████████████████| 323 kB 19.0 MB/s eta 0:00:01
     |████████████████████████████████| 8.3 MB 24.0 MB/s eta 0:00:01
     |████████████████████████████████| 23.1 MB 40.6 MB/s eta 0:00:01
     |████████████████████████████████| 17.3 MB 16.1 MB/s eta 0:00:01


In [1]:
from google.maps import places_v1
import pandas as pd

In [3]:
client = places_v1.PlacesClient()
input_data = pd.read_csv("./input_data/organizations_with_no_info.csv")
organizations = input_data['organization'].tolist()

In [4]:
addresses = []
cities = []
latitudes = []
longitudes = []
types = []
display_names = []
primary_types = []
phone_numbers = []
websites = []
for organization in organizations:
    request = places_v1.SearchTextRequest(
        text_query = organization + 'Region of Waterloo',
    )
    response = client.search_text(request=request, 
                                  metadata=[("x-goog-fieldmask", 
                                             "places.displayName,places.addressComponents,places.formattedAddress,places.location,places.types,places.displayName,places.primaryType,places.nationalPhoneNumber,places.websiteUri")])
    if len(response.places) > 0:
        addresses.append(response.places[0].formatted_address)
        longitudes.append(response.places[0].location.longitude)
        latitudes.append(response.places[0].location.latitude)
        types.append(response.places[0].types)
        display_names.append(response.places[0].display_name)
        primary_types.append(response.places[0].primary_type)
        phone_numbers.append(response.places[0].national_phone_number)
        websites.append(response.places[0].website_uri)
        address_details = response.places[0].address_components
        city = "Unknown"
        for i in address_details:
            if i.types[0] == 'locality':
                city = i.long_text
        cities.append(city)
    else:
        addresses.append('NaN')
        longitudes.append('NaN')
        latitudes.append('NaN')
        types.append('NaN')
        display_names.append('NaN')
        primary_types.append('NaN')
        phone_numbers.append('NaN')
        websites.append('NaN')
        cities.append('NaN')
location_data = pd.DataFrame.from_dict({'org' : organizations,
                                        'AI_formatted_name' : display_names,
                                        'AI_types' : types,
                                        'AI_primary_type' : primary_types,
                                        'AI_website' : websites,
                                        'AI_phone' : phone_numbers,
                                        'AI_address' : addresses,
                                        'AI_city' : cities,
                                        'AI_longitude' : longitudes,
                                        'AI_latitude' : latitudes
                                       })
location_data

,org,AI_formatted_name,AI_types,AI_primary_type,AI_website,AI_phone,AI_address,AI_city,AI_longitude,AI_latitude
0,Focus for Ethnic Women,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Cambridge Newcomer Outreach Program,"text: ""Welcome Centre for Newcomers""\nlanguage...","[school, point_of_interest, establishment]",school,https://www.wrdsb.ca/register/welcome-centre-f...,(519) 576-3018,"325 Louisa St, Kitchener, ON N2H 5K9, Canada",Kitchener,-80.490161,43.459355
2,Waterloo Regional Housing,"text: ""Waterloo Regional Housing""\nlanguage_co...","[point_of_interest, establishment]",,,(519) 749-2471,"119 College St, Kitchener, ON N2H 6M5, Canada",Kitchener,-80.490662,43.454584
3,44 Gaukel,"text: ""44 Gaukel St""\n",[street_address],,,,"44 Gaukel St, Kitchener, ON N2G 4P3, Canada",Kitchener,-80.493479,43.449748
4,Migrant Workers Action Campaign,"text: ""Workers Action Center""\nlanguage_code: ...","[point_of_interest, establishment]",,http://workersactioncentre.org/,(416) 531-0778,"720 Spadina Ave., Toronto, ON M5S 2T9, Canada",Toronto,-79.403378,43.665406
...,...,...,...,...,...,...,...,...,...,...
150,Forest Heights Community Association,"text: ""Forest Heights Community Centre""\nlangu...","[community_center, event_venue, point_of_inter...",community_center,http://www.kitchener.ca/forestheightscc/,(519) 741-2621,"1700 Queen's Blvd, Kitchener, ON N2N 3L6, Canada",Kitchener,-80.527816,43.427753
151,Grand Hill Village Neighbourhood Association,"text: ""Hespeler Village Neighbourhood Associat...","[point_of_interest, establishment]",,http://hvna.ca/,(519) 240-3567,"640 Ellis Rd, Cambridge, ON N3C 4N1, Canada",Cambridge,-80.291735,43.428018
152,Green Belt Neighbourhood Association,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153,Lower Doon Neighbourhood Association,"text: ""Lower Doon""\nlanguage_code: ""en""\n","[neighborhood, political]",,,,"Lower Doon, Kitchener, ON, Canada",Kitchener,-80.414395,43.392205


In [5]:
location_data.to_csv('output_data/organizations_with_no_info_supplement.csv', index = False)

In [8]:
input_data['AI_address'] = addresses
input_data['AI_longitude'] = longitudes
input_data['AI_latitude'] = latitudes
input_data['AI_types'] = types
input_data['AI_formatted_name'] = display_names
input_data['AI_primary_type'] = primary_types
input_data['AI_phone'] = phone_numbers
input_data['AI_website'] = websites
input_data['AI_city'] = cities
input_data

,organization,AI_address,AI_longitude,AI_latitude,AI_types,AI_formatted_name,AI_primary_type,AI_phone,AI_website,AI_city
0,Focus for Ethnic Women,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Cambridge Newcomer Outreach Program,"325 Louisa St, Kitchener, ON N2H 5K9, Canada",-80.490161,43.459355,"[school, point_of_interest, establishment]","text: ""Welcome Centre for Newcomers""\nlanguage...",school,(519) 576-3018,https://www.wrdsb.ca/register/welcome-centre-f...,Kitchener
2,Waterloo Regional Housing,"119 College St, Kitchener, ON N2H 6M5, Canada",-80.490662,43.454584,"[point_of_interest, establishment]","text: ""Waterloo Regional Housing""\nlanguage_co...",,(519) 749-2471,,Kitchener
3,44 Gaukel,"44 Gaukel St, Kitchener, ON N2G 4P3, Canada",-80.493479,43.449748,[street_address],"text: ""44 Gaukel St""\n",,,,Kitchener
4,Migrant Workers Action Campaign,"720 Spadina Ave., Toronto, ON M5S 2T9, Canada",-79.403378,43.665406,"[point_of_interest, establishment]","text: ""Workers Action Center""\nlanguage_code: ...",,(416) 531-0778,http://workersactioncentre.org/,Toronto
...,...,...,...,...,...,...,...,...,...,...
150,Forest Heights Community Association,"1700 Queen's Blvd, Kitchener, ON N2N 3L6, Canada",-80.527816,43.427753,"[community_center, event_venue, point_of_inter...","text: ""Forest Heights Community Centre""\nlangu...",community_center,(519) 741-2621,http://www.kitchener.ca/forestheightscc/,Kitchener
151,Grand Hill Village Neighbourhood Association,"640 Ellis Rd, Cambridge, ON N3C 4N1, Canada",-80.291735,43.428018,"[point_of_interest, establishment]","text: ""Hespeler Village Neighbourhood Associat...",,(519) 240-3567,http://hvna.ca/,Cambridge
152,Green Belt Neighbourhood Association,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153,Lower Doon Neighbourhood Association,"Lower Doon, Kitchener, ON, Canada",-80.414395,43.392205,"[neighborhood, political]","text: ""Lower Doon""\nlanguage_code: ""en""\n",,,,Kitchener


In [9]:
input_data.to_csv('output_data/organizations_with_no_info_augmented.csv', index = False)